In [2]:
from openai import AssistantEventHandler
from typing_extensions import override
from openai import OpenAI

In [49]:
client = OpenAI(api_key="")

In [8]:

assistant = client.beta.assistants.create(
    name="Math Tutor 2",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o",
)


In [12]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print([f"assitant {a.id} and name: {a.name}" for a in my_assistants.data])

['assitant asst_6uGjzjpWWsDxji48eR4swmVf and name: Math Tutor 2', 'assitant asst_WMDfBOquGuD2ps4xXa5ZDU5Z and name: Math Tutor', 'assitant asst_6ZAy6JOCYLdI2WU3UNHfaU8c and name: Math Tutor', 'assitant asst_tH2T6XSwVFGNe7isUss4QSw2 and name: Math Tutor', 'assitant asst_ZffcrFABeWzbhFQ3ukBXO3bB and name: Math Tutor', 'assitant asst_ifPrzt2p7RLa2rjaqdgu1e3N and name: Math Tutor', 'assitant asst_nrEykEJvyeAnO2s8ZVCAYCVo and name: Math Tutor', 'assitant asst_tPgEDd5kX1c3tX9rgFfae2vT and name: None', 'assitant asst_cpHUsNuKNC1pIJHZkDh2RBTl and name: None', 'assitant asst_x77oShKZ6B3LPs5wWWGwKQ5b and name: None']


In [5]:

thread = client.beta.threads.create()

In [42]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)


# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.


with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()




assistant > Absolutely, Jane Doe! Let's go through it step by step one more time.

Given the equation:
\[ 3x + 11 = 14 \]

1. **Subtract 11 from both sides:**
   \[ 3x + 11 - 11 = 14 - 11 \]
   Which simplifies to:
   \[ 3x = 3 \]

2. **Divide both sides by 3:**
   \[ \frac{3x}{3} = \frac{3}{3} \]
   Which simplifies to:
   \[ x = 1 \]

Thus, the solution to the equation is \( x = 1 \). If you have any further questions or need anything else, let me know!

In [13]:
class StreamEventHandler(AssistantEventHandler):
    # Iterator over just the text deltas in the stream
    @override
    def on_text_created(self, text) -> None:
        # Callback that is fired when a text content block is created
        # print(f"\nassistant > ", end="", flush=True)
        pass

    @override
    def on_text_delta(self, delta, snapshot):
        """Callback that is fired whenever a text content delta is returned
        by the API.

        The first argument is just the delta as sent by the API and the second argument
        is the accumulated snapshot of the text. For example:

        on_text_delta(TextDelta(value="The"), Text(value="The")),
        on_text_delta(TextDelta(value=" solution"), Text(value="The solution")),
        on_text_delta(TextDelta(value=" to"), Text(value="The solution to")),
        on_text_delta(TextDelta(value=" the"), Text(value="The solution to the")),
        on_text_delta(TextDelta(value=" equation"), Text(value="The solution to the equivalent")),
        """
        # print(delta.value, end="", flush=True)
        pass

In [47]:
stream = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  stream=True
)

for event in stream:
  print(event)
  if event.event != "thread.message.delta":
    continue

  # for content_delta in event.data.delta.content or []:
  #   if content_delta.type == "text" and content_delta.text and content_delta.text.value:
  #       print(content_delta.text.value)
  print(event.data.delta.content[0].text.value)


ThreadRunCreated(data=Run(id='run_jo4CfCPaAwrf3bqwbuKXnVe1', assistant_id='asst_6uGjzjpWWsDxji48eR4swmVf', cancelled_at=None, completed_at=None, created_at=1719903085, expires_at=1719903685, failed_at=None, incomplete_details=None, instructions='You are a personal math tutor. Write and run code to answer math questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Kgxf815ZsIOCN0YOOkGyjYwI', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={'code_interpreter': {'file_ids': []}}), event='thread.run.created')
ThreadRunQueued(data=Run(id='run_jo4CfCPaAwrf3bqwbuKXnVe1', assistant_id='asst_6uGjzjpWWsDxji48eR4swmVf', cancelled_at=None, completed_

In [37]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
) as stream:
    if stream.on_text_created:
        print('@@@@ start the big bang')
    if stream.on_text_delta:
        # for text in stream.text_deltas:
        # print(*stream.text_deltas, end="", flush=True)
        print('-------')
        print(stream.)
    if stream.on_end:
        print('####syaonara')

@@@@ start the big bang
-------
<generator object AssistantEventHandler.__text_deltas__ at 0x10ce71890>
####syaonara


In [48]:
assistant = client.beta.assistants.retrieve(
                "session.assistant_id")

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'session.assistant_id'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}